# CS 3020: Compiler Construction

## In-class Exercises, Week of 01/13/2025

----
# PART I: Abstract Syntax Trees

## Question 1

The following grammar defines the *concrete syntax* for a language of integer arithmetic (numbers and the "plus" operator).

\begin{align*}
expr &::= n \\
&\mid expr + expr
\end{align*}

The following class hierarchy defines an *abstract syntax* for the same language.

In [3]:
from dataclasses import dataclass
from cs3020_support.python import AST, print_ast

@dataclass
class Expr(AST):
    pass

@dataclass
class Constant(Expr):
    val: int

@dataclass
class Plus(Expr):
    e1: Expr
    e2: Expr

Write an abstract syntax tree for the expression `1 + 2 + 3`.

In [12]:
Plus(Plus(Constant(1), Constant(2)), Constant(3))

Plus(e1=Plus(e1=Constant(val=1), e2=Constant(val=2)), e2=Constant(val=3))

In [13]:
ast = Plus(Plus(Constant(1), Constant(2)), Constant(3))
print(print_ast(ast))

Plus(
 Plus(
  Constant(1),
  Constant(2)),
 Constant(3))


## Question 2

The code below defines a parser that transforms concrete syntax for this simple language into abstract syntax trees.

In [14]:
from lark import Lark
_rint_parser = Lark(r"""
    ?exp: NUMBER -> int_e
        | exp "+" exp -> plus_e
        | "(" exp ")"
    %import common.NUMBER
    %import common.CNAME
    %import common.WS
    %ignore WS
    """, start='exp')

def parse(s):
    def t_ast(e):
        if e.data == 'int_e':
            return Constant(int(e.children[0]))
        elif e.data == 'plus_e':
            e1, e2 = e.children
            return Plus(t_ast(e1), t_ast(e2))

    parsed = _rint_parser.parse(s)
    ast = t_ast(parsed)
    return ast

In [17]:
_rint_parser.parse('1+2+3')

Tree('plus_e', [Tree('plus_e', [Tree('int_e', [Token('NUMBER', '1')]), Tree('int_e', [Token('NUMBER', '2')])]), Tree('int_e', [Token('NUMBER', '3')])])

Write code to use the parser above to parse the expression `1 + 2 + 3` into an abstract syntax tree.

In [18]:
ast = parse('1+2+3')
print(print_ast(ast))

Plus(
 Plus(
  Constant(1),
  Constant(2)),
 Constant(3))


## Question 3

Write an *interpreter* for this language.

**The structure of your function should follow the structure of the AST**

In [24]:
def eval_rint(e: Expr) -> int:
    match e:   # 1. match statement on the input AST
        case Constant(i): # 2. add a match case for each possible AST node of that type
            # variable i is bound to e.val
            return i
        case Plus(e1, e2):
            # variable e1 bound to e.e1
            v1 = eval_rint(e1) # 3. add recursive calls for subexpressions
            v2 = eval_rint(e2)
            return v1 + v2     # 4. thinking

In [25]:
# TEST CASE
assert eval_rint(parse('1 + 2 + 3')) == 6
assert eval_rint(parse('42 + 20 + 10 + 5 + 5')) == 82

----
# PART II: x86 Assembly

In [26]:
from cs3020_support.eval_x86 import X86Emulator

## Question 4

Write x86 assembly code to add the numbers 1 and 2, putting the result in the register `rax`.

In [30]:
emu = X86Emulator(logging = False)
emu.eval_instructions("""
movq $1, %rax
addq $2, %rax""")

,Location,Old,New
0,reg rax,None,3


## Question 5 

Write x86 assembly code to add the numbers 1, 2, 3, and 4, putting the result in the register `rdi`.

In [32]:
emu = X86Emulator(logging = False)
emu.eval_instructions("""
movq $1, %rax
addq $2, %rax
addq $3, %rax
addq $4, %rax
movq %rax, %rdi""")

,Location,Old,New
0,reg rax,None,10
1,reg rdi,None,10


## Question 6

Write a complete x86 program to:

- Place the number 42 in the register `rdi`
- Call the function `print_int` in the runtime
- Return cleanly to the operating system

Hint: try using the [Assignment 1 online compiler](https://jnear.w3.uvm.edu/cs3020/compiler-a1.php).

In [35]:
emu = X86Emulator(logging = False)
emu.eval_program("""
.globl main
main:
  movq $42, %rdi
  callq print_int
  retq""")

[42]

## Question 7

Write code to generate a *pseudo-x86 abstract syntax tree* for the `main` block in the program above.

Hint: reference the [pseudo-x86 AST class hierarchy](https://github.com/jnear/cs3020-assignments/blob/main/cs3020_support/x86.py). Debug your solution using the online compiler's output for the `select instructions` pass.

In [38]:
import cs3020_support.x86 as x86

ast = x86.X86Program( # x86 program is a dict mapping labels to lists of instrs
 {
  'main':
   [
    x86.Movq(
     x86.Immediate(42), # encodes $42
     x86.Reg("rdi")),   # encodes %rdi
    x86.Callq("print_int"), # encodes callq print_int
    x86.Retq()              # encodes retq
   ]
 })

print(print_ast(ast))

X86Program(
 {
  'main':
   [
    Movq(
     Immediate(42),
     Reg("rdi")),
    Callq("print_int"),
    Retq()
   ]
 },
 None)


## Question 8

What is the purpose of the `select_instructions` pass of the compiler? How should it be implemented?

- The pass transforms a Python program AST into an X86 assembly language AST
- It works by matching on the structure of the input Python program, then returning a new X86 program AST